## Demo

In [ ]:
from wavescapes import *
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
#from utils import *
import pandas as pd
import json
from scipy import ndimage

In [ ]:
scores_muse = glob('./MS3/*.mscx') #to understand how to make musescores work

In [ ]:
json_job = []

for score in scores_muse:
    json_job.append({
    "in": score,
    "out": './midi\\' + score.split('\\')[-1][:-5] + '.mid',
  })

#print(json_job)

In [ ]:
print(glob('./*'))

In [ ]:
import os
if '.\\midi' not in glob('./*'):
    print('making folder')
    os.makedirs('midi')
    

In [ ]:
with open('json_job.json', 'w') as outfile:
    json.dump(json_job, outfile)

In [ ]:
! MuseScore3.exe -j json_job.json

In [ ]:
data = pd.read_csv('./metadata.tsv', sep='\t')
data.head()

In [ ]:
data.columns # to use later especially file name for merging, duration, key, composed start and composed end

In [ ]:
scores = glob('./midi/*.mid')
print(len(scores))

In [ ]:
score_names = pd.Series(scores, index=[score.split('\\')[-1].split('.')[0] for score in scores], name='filepath')
score_names.head()

In [ ]:
data = data.merge(score_names, left_on='fnames', right_index=True)
data.head()

In [ ]:
from utils import *

utm_max, utm_entropy, utm_argmax, utm = most_resonant(data['filepath'][1], aw_size=4)
utm_max.shape

In [ ]:
utm_max

In [ ]:
utm_entropy

In [ ]:
utm_sum = max_utm_to_ws_utm(utm_max, utm_argmax, utm, 'max')
utm_sum = max_utm_to_ws_utm(utm_entropy, utm_argmax, utm, 'entropy')

In [ ]:
x, y = center_of_mass_v(utm_max, utm)
print(x, y)

In [ ]:
Wavescape(utm_sum, 500).draw()
plt.plot(x, y, marker="o")
print(data['fnames'][1])

In [ ]:
for i in range(1,7):
    x, y = center_of_mass_v(np.abs(utm[:,:,i]), utm)
    print(x, y)


In [ ]:
coeff_5 = complex_utm_to_ws_utm(utm, coeff=5)
x, y = center_of_mass_v(np.abs(utm[:,:,5]), utm)
Wavescape(coeff_5, 500).draw()
plt.plot(x, y, marker="o")
print(x, y)
print(data['fnames'][1])

In [ ]:
all_wavescapes(data['filepath'][1], individual_width=500, aw_size=4)

In [ ]:
if '.\\figures' not in glob('./*'):
    print('making folder')
    os.makedirs('figures')


In [ ]:
import matplotlib.patches as mpatches
patches = []
for i in range(6):
    patches.append(mpatches.Patch(color=np.array(circular_hue_revised(i))[:3]/255, label='Coefficient '+str(i+1)))
        
for score in data['filepath']:
    print(str(score.split('\\')[-1].split('.')[0]))
    try:
        #all_wavescapes(score, individual_width=500, aw_size=4, aw_per_tick=1, tick_start=1)
        utm_max, utm_entropy, utm_argmax, utm = most_resonant(score, aw_size=4)
        utm_sum = max_utm_to_ws_utm(utm_max, utm_argmax, utm, 'max')
        Wavescape(utm_sum, 500).draw()
        plt.suptitle(str(score.split('\\')[-1].split('.')[0]) + '\n', fontsize=14)
        plt.legend(handles=patches)
        plt.savefig('./figures/' + str(score.split('\\')[-1].split('.')[0]) + '.png')
    except:
        print('Not successful')

In [ ]:
import matplotlib.patches as mpatches
patches = []
for i in range(6):
    patches.append(mpatches.Patch(color=np.array(circular_hue_revised(i))[:3]/255, label='Coefficient '+str(i+1)))

for score in data['filepath']:
    print(str(score.split('\\')[-1].split('.')[0]))
    try:
        #all_wavescapes(score, individual_width=500, aw_size=4, aw_per_tick=1, tick_start=1)
        utm_max,  utm_entropy, utm_argmax, utm = most_resonant(score, aw_size=4)
        utm_sum = max_utm_to_ws_utm(utm_entropy, utm_argmax, utm, 'entropy')
        Wavescape(utm_sum, 500).draw()
        plt.suptitle('entropy - ' + str(score.split('\\')[-1].split('.')[0]) + '\n', fontsize=14)
        plt.legend(handles=patches)
        plt.savefig('./figures/entropy_' + str(score.split('\\')[-1].split('.')[0]) + '.png')
    except Exception as e:
        print(e)

In [ ]:
# legend
import webcolors

for angle in range(6):

    value = np.array(circular_hue_revised(angle))[:3]
    print('coefficient', angle+1, ':',  webcolors.rgb_to_name(value))

In [ ]:
vcoms = []
for it in data.iterrows():
    print(it[1]['filepath'])
    utm_max, utm_argmax, utm = most_resonant(it[1]['filepath'], aw_size=4)
    vcoms.append(center_of_mass_v(utm_max, utm)[1])

In [ ]:
data['VCOM'] = vcoms

In [ ]:
plt.scatter(data['composed_start'], data['VCOM'])

In [ ]:
plt.scatter(data['composed_end'], data['VCOM'])

In [ ]:
from utils import *
utm_max, utm_entropy, utm_argmax, utm = most_resonant(data['filepath'][1], aw_size=4)
arr1 = produce_pitch_class_matrix_from_filename(data['filepath'][1], aw_size=4)

arr1.shape

In [ ]:
utm_tritone = pitch_class_matrix_to_tritone(arr1)


In [ ]:
utm_tritone.shape
utm_tritone = np.mean(utm_tritone, axis=2)
utm_tritone[utm_tritone != 0.] = 1

In [ ]:
utm_tritone.shape
utm_tritone

In [ ]:
arr1[0]

In [ ]:
arr2 = np.roll(arr1, 6, axis=1)
arr2[0]

In [ ]:
tritone = arr1 * arr2
tritone[4]

In [ ]:
utm.shape

In [ ]:
utm = np.concatenate((utm, utm_tritone.reshape((71,71,1))), axis=2)
utm.shape